# NPRG030 Programování I – cvičení 4.12.2024

### Problem: bludiste
```
  ######
# #    #
# ##   #
#  # # #
#    # #
######  
```

In [ ]:
from typing import Callable


def get_neighbours(pos, size):
    """return positions of valid neighbors"""
    ns = []
    for delta in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        x = pos[0] + delta[0]
        y = pos[1] + delta[1]
        if 0 <= x < size[0] and 0 <= y < size[1]:
            ns.append((x, y))
    return ns

def bfs(maze, visited, back, start, end):
    size = (len(maze), len(maze[0]))
    queue = [start]
    while queue:
        node = queue.pop(0)
        if visited[node[0]][node[1]]:
            continue
        visited[node[0]][node[1]] = 1
        if node == end:
            break
        for neighbour in get_neighbours(node, size):
            if visited[neighbour[0]][neighbour[1]] == 0 and maze[neighbour[0]][neighbour[1]] == 1:
                back[neighbour[0]][neighbour[1]] = node
                queue.append(neighbour)

obecne:

pocatecni stav, konecny stav

funkce, ktera mi z momentalniho stavu da sousedni stavy

funkce, ktera rozhodne o ekvivalenci dvou stavu

In [23]:
from collections import deque
from typing import Callable


# obecna implementace prohledavani stavoveho prostoru
def solve(start, end, neigh_fn: Callable, state_eq: Callable, state_key: Callable):
    visited = {}
    back = {}
    q = deque([start])
    found = False
    while q:
        state = q.popleft()
        visited[state_key(state)] = True
        if state_eq(state, end):
            found = True
            break
        
        for neighbour in neigh_fn(state):
            if visited.get(state_key(neighbour), False):
                continue
            back[state_key(neighbour)] = state
            q.append(neighbour)
    
    if found:
        b = end
        path = []
        while b != start:
            path.append(b)
            b = back[state_key(b)]
        path.append(start)
        path.reverse()
        return path
    return [None]


In [15]:
# konkretni implementace funkci neigh_fn, state_eq, state_key 
# pro problem skakani kone pro sachovnici

def knight_neigh_fn(state):
    state_x, state_y = state
    neigh = []
    for dx, dy in [( 2, 1), ( 2, -1),
                   (-2, 1), (-2, -1),
                   ( 1, 2), ( 1, -2),
                   (-1, 2), (-1, -2)]:
        x = state_x + dx
        y = state_y + dy
        if 1 <= x <= 8 and 0 <= y <= 8:
            neigh.append((x, y))
    return neigh

def state_eq(state1, state2):
    return state1[0] == state2[0] and state1[1] == state2[1]
    # nebo proste... return state1 == state2

def state_key(state):
    return state[0] * 10 + state[1]

print(solve(start=(1, 1),
            end=(4, 6),
            neigh_fn=knight_neigh_fn,
            state_eq=state_eq,
            state_key=state_key))


[(1, 1), (2, 3), (1, 5), (2, 7), (4, 6)]


In [12]:
# zmena:
# sudy tah
# lichy tah ... pouze o policko nahoru

In [17]:
def knight_neigh_fn(state):
    state_x, state_y, state_parity = state
    neigh = []
    if state_parity % 2 == 0:
        for dx, dy in [( 2, 1), ( 2, -1),
                       (-2, 1), (-2, -1),
                       ( 1, 2), ( 1, -2),
                       (-1, 2), (-1, -2)]:
            x = state_x + dx
            y = state_y + dy
            if 1 <= x <= 8 and 0 <= y <= 8:
                neigh.append((x, y, state_parity + 1))
    else:
        if 1 <= state_y + 1 <= 8:
            neigh.append((state_x, state_y + 1, state_parity + 1))
    return neigh

def state_eq(state1, state2):
    return state1[0] == state2[0] and state1[1] == state2[1]
    # nebo proste... return state1 == state2

def state_key(state):
    return state[0] * 10 + state[1]

print(solve(start=(1, 1, 0),
            end=(4, 6, "x"),
            neigh_fn=knight_neigh_fn,
            state_eq=state_eq,
            state_key=state_key))

[(1, 1, 0), (2, 3, 1), (2, 4, 2), (4, 5, 3), (4, 6, 'x')]


In [34]:
# prelevani mezi kbeliky A, B, C
# info:
# pocatecni objem vody v kazdem kbeliku
# cilovy objem v kazdem kbeliku
# apacita kbeliku
# pravidla:
# bud kbelik uplne vyleju do dalsiho
# nebo liji do maximalni kapacity ciloveho kbeliku

state = (("objem v A", "kapacita A"),
         ("objem v B", "kapacita B"), 
         ("objem v C", "kapacita C"),
         "X->X")

def get_neigh_fn(state):
    a, max_a = state[0]
    b, max_b = state[1]
    c, max_c = state[2]
    
    neigh = []
    if a > 0:
        if b + a <= max_b:
            neigh.append(((0, max_a), (b + a, max_b), (c, max_c), "A->B"))
        elif b != max_b:
            neigh.append(((a - (max_b - b), max_a), (max_b, max_b), (c, max_c), "A->B"))
        if c + a <= max_c:
            neigh.append(((0, max_a), (b, max_b), (c + a, max_c), "A->C"))
        elif c != max_c:
            neigh.append(((a - (max_c - c), max_a), (b, max_b), (max_c, max_c), "A->C"))
    if b > 0:
        if a + b <= max_a:
            neigh.append(((a + b, max_a), (0, max_b), (c, max_c), "B->A"))
        elif a != max_a:
            neigh.append(((max_a, max_a), (b - (max_a - a), max_b), (c, max_c), "B->A"))
        if c + b <= max_c:
            neigh.append(((a, b), (0, max_b), (c + b, max_c), "B->C"))
        elif c != max_c:
            neigh.append(((a, max_a), (b - (max_c - c), max_b), (max_c, max_c), "B->C"))
    if c > 0:
        if a + c <= max_a:
            neigh.append(((a + c, max_a), (b, max_b), (0, max_c), "C->A"))
        elif a != max_a:
            neigh.append(((max_a, max_a), (b, max_b), (c - (max_a - a), max_c), "C->A"))
        if b + c <= max_b:
            neigh.append(((a, max_a), (b + c, max_b), (0, max_c), "C->B"))
        elif b != max_b:
            neigh.append(((a, max_a), (max_b, max_b), (c - (max_b - b), max_c), "C->B"))
        
    return neigh

def state_eq(state1, state2):
    return state1[0][0] == state2[0][0] and state1[1][0] == state2[1][0] and state1[2][0] == state2[2][0]

def state_key(state):
    return state[0][0], state[1][0], state[2][0]

s = solve(
    ((5, 10), (3, 7), (4, 5), ""),
    ((10, 10), (2, 7), (0, 5), ""),
    get_neigh_fn,
    state_eq,
    state_key
)
print(*map(lambda x: (x[0][0], x[1][0], x[2][0], x[3]), s), sep="\n")
    

(5, 3, 4, '')
(1, 7, 4, 'A->B')
(8, 0, 4, 'B->A')
(8, 4, 0, 'C->B')
(10, 2, 0, '')
